# **Grasping Siamese Mask R-CNN - Training on the OCID Grasp Dataset**

This notebook provides a demonstration on how to train the Grasping Siamese Mask R-CNN (GSMR-CNN) model on the __[OCID grasp](https://github.com/stefan-ainetter/grasp_det_seg_cnn)__ dataset. The OCID grasp dataset is an extension of the __[OCID](https://www.acin.tuwien.ac.at/en/vision-for-robotics/software-tools/object-clutter-indoor-dataset/)__ dataset which was designed for robotic vision tasks that includes object-segmentation, classification and recognition. It contains RGBD scenes of cluttered objects where each objects is annotated with a class and segmentation map. The grasping extension adds multiple hand-annotated grasp candidates for each objects.

The code below adds the python modules (i.e. `model.py`, `utils.py`) needed to build the dataloader and construct the GSMR-CNN model. The code used to build GSMR-CNN overwrites methods that were used to build both __[Siamese Mask R-CNN](https://github.com/bethgelab/siamese-mask-rcnn)__ and __[Mask R-CNN](https://github.com/matterport/Mask_RCNN)__, hence, both libraries need to be included in the project directory. This snippet also instantiates a configuration object, which uses the default variables specified in `config.py`. 

* Before running this folder, make sure to set the variables `DATASET_PATH` and `ANNOTATIONS_PATH` to the directory paths of the OCID grasp dataset and the folder where the COCO-formatted annotations of the OCID grasp dataset were placed. Note that if you do not have the annotations, it is important you ran the `generate_COCO.py` file to generate COCO-formatted annotations before running this code as specified in the `ReadMe` file. Make sure you also downloaded the ImageNet weights into the `models/` directory.

In [1]:
# ignore FutureWarnings and Tensorflow warnings
import warnings
import os
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)

# add paths to all libraries
import sys
sys.path.append('libraries/')
sys.path.append('libraries/Mask_RCNN') 
sys.path.append('libraries/Siamese_Mask_RCNN') 
sys.path.append('gsmrcnn/')

import model as gsmrcnn_model
import config as gsmrcnn_config
import utils as gsmrcnn_utils

from sklearn.model_selection import train_test_split
import numpy as np
from collections import OrderedDict

config = gsmrcnn_config.OCIDConfig()

DATASET_PATH = 'data/OCID_grasp/'
ANNOTATIONS_PATH = 'data/'

Using TensorFlow backend.


## **Dataset**

The code in this section loads and prepares the dataset for training. It also selects which object classes will be used for training and testing the model. Note that certain object classes were excluded from both training and testing set because they were missing grasp annotations mainly due to them being large objects (e.g. keyboard).

* The code below is used to train the model on an **object-wise split**, however, to train it on an image-wise split the lines `train_classes = all_classes` and `test_classes = all_classes` should be uncommented and used instead. 


In [2]:
ignore_classes = [5, 7, 11, 12, 16, 17] # exclude classes due to abundant missing grasps...

# for object-wise split
all_classes = np.array([i for i in range(1,32) if i not in ignore_classes])
train_classes, test_classes = train_test_split(all_classes, test_size=0.30, random_state=10)

# for image-wise split (uncomment lines below if needed)
# train_classes = all_classes
# test_classes = all_classes

# load training dataset
coco_train = gsmrcnn_utils.OCIDDataset()
coco_train.load_coco(os.path.join(ANNOTATIONS_PATH, 'training_annotations.json'), DATASET_PATH)
coco_train.prepare()
coco_train.build_indices() # creates lists mapping object classes ids to image ids and vice versa
coco_train.ACTIVE_CLASSES = train_classes

# load validation dataset
coco_val = gsmrcnn_utils.OCIDDataset()
coco_val.load_coco(os.path.join(ANNOTATIONS_PATH, 'validation_annotations.json'), DATASET_PATH)
coco_val.prepare()
coco_val.build_indices() # creates lists mapping object classes ids to image ids and vice versa
coco_val.ACTIVE_CLASSES = test_classes

print("[INFO] Number of training images:", len(coco_train.image_category_index))
print("[INFO] Number of validation images:", len(coco_val.image_category_index))

loading annotations into memory...
Done (t=1.65s)
creating index...
index created!
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
[INFO] Number of training images: 1411
[INFO] Number of validation images: 352


## **Model**

The code below is used to construct the GSMR-CNN model with pre-trained weights from ImageNet. The model is trained for 25 epochs and a scheduler is used to train the head components of the model for the first epoch and the entire model for the remaining epochs, having a lower learning rate for the last five epochs. 

* Note that it will create a directory in `{cfg.CHECKPOINT_DIR}/gsmrcnn_{cfg.NAME}_{cfg.EXPERIMENT}` to store training checkpoints under.

In [3]:
# create model and load weights trained on Imagenet
model = gsmrcnn_model.GraspingSiameseMaskRCNN(mode="training", model_dir=config.CHECKPOINT_DIR, config=config)
model.load_imagenet_weights(pretraining='imagenet-687')

# create a scheduler
train_schedule = OrderedDict()
train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
train_schedule[20] = {"learning_rate": config.LEARNING_RATE, "layers": "all"}
train_schedule[25] = {"learning_rate": config.LEARNING_RATE / 10, "layers": "all"}

initializing from imagenet weights ...


# **Training**

The code below can be run to train the network. 

* Note that the saved model will be saved as `{cfg.CHECKPOINT_DIR}/gsmrcnn_{cfg.NAME}_{cfg.EXPERIMENT}/gsmrcnn_0025.h5`. Additionally, the output below is output obtained from the terminal when training one of the models during experimentations.

In [5]:
for epochs, parameters in train_schedule.items():
    print("training layers {} until epoch {} with learning_rate {}".format(parameters["layers"], epochs, parameters["learning_rate"]))
    model.train(coco_train, coco_val,
                learning_rate=parameters["learning_rate"],
                epochs=epochs,
                layers=parameters["layers"])

training layers heads until epoch 1 with learning_rate 0.02

Starting at epoch 0. LR=0.02

Checkpoint Path: models/gsmrcnn_object_split_1\gsmrcnn_{epoch:04d}.h5
Epoch 1/1
  150/150 [==============================] - 2070s 14s/step - loss: 1.7335 - rpn_class_loss: 0.3302 - rpn_bbox_loss: 0.1097 - mrcnn_class_loss: 0.2470 - mrcnn_bbox_loss: 0.3087 - mrcnn_mask_loss: 0.5859 - mrcnn_grasp_loss: 0.1519 - val_loss: 1.1943 - val_rpn_class_loss: 0.1087 - val_rpn_bbox_loss: 0.0720 - val_mrcnn_class_loss: 0.2554 - val_mrcnn_bbox_loss: 0.2312 - val_mrcnn_mask_loss: 0.4575 - val_mrcnn_grasp_loss: 0.0694
training layers all until epoch 20 with learning_rate 0.02

Checkpoint Path: models/gsmrcnn_object_split_1\gsmrcnn_{epoch:04d}.h5
Epoch 2/20
150/150 [==============================] - 2357s 16s/step - loss: 1.1893 - rpn_class_loss: 0.0958 - rpn_bbox_loss: 0.0614 - mrcnn_class_loss: 0.3289 - mrcnn_bbox_loss: 0.2038 - mrcnn_mask_loss: 0.4433 - mrcnn_grasp_loss: 0.0558 - val_loss: 0.9978 - val_rpn_cla